In [1]:
import findspark
findspark.init()
import pyspark
import pyspark.sql
from pyspark import SparkConf
from pyspark import SparkContext
conf = SparkConf()#
conf.set("spark.python.profile", "true")
#conf = sc.getConf()
conf.set( "spark.driver.memory", "4g" )
conf.set( "spark.jars","/Users/crisliu/agile/lib/mongo-hadoop-spark-2.0.2.jar,\
/Users/crisliu/agile/lib/mongo-java-driver-3.4.0.jar,\
/Users/crisliu/agile/lib/elasticsearch-spark-20_2.10-5.0.0-alpha5.jar,\
/Users/crisliu/agile/lib/snappy-java-1.1.2.6.jar,\
/Users/crisliu/agile/lib/lzo-hadoop-1.0.5.jar")
#/Users/crisliu/agile/lib/elasticsearch-hadoop-5.0.0-alpha5.jar"
#conf.set("spark.executor.extraClassPath","~")
sc = SparkContext('local', 'test', conf=conf)
#sc = pyspark.SparkContext()
APP_NAME = "my_script.py"
spark = pyspark.sql.SparkSession(sc).builder.appName(APP_NAME).getOrCreate()

In [4]:
base_path = '/Users/crisliu/agile/'

In [6]:
#
# {
#   "ArrDelay":5.0,"CRSArrTime":"2015-12-31T03:20:00.000-08:00","CRSDepTime":"2015-12-31T03:05:00.000-08:00",
#   "Carrier":"WN","DayOfMonth":31,"DayOfWeek":4,"DayOfYear":365,"DepDelay":14.0,"Dest":"SAN","Distance":368.0,
#   "FlightDate":"2015-12-30T16:00:00.000-08:00","FlightNum":"6109","Origin":"TUS"
# }
#
from pyspark.sql.types import StringType, IntegerType, FloatType, DoubleType, DateType, TimestampType
from pyspark.sql.types import StructType, StructField
from pyspark.sql.functions import udf

schema = StructType([
StructField("ArrDelay", DoubleType(), True),     # "ArrDelay":5.0
StructField("CRSArrTime", TimestampType(), True),    # "CRSArrTime":"2015-12-31T03:20:00.000-08:00"
StructField("CRSDepTime", TimestampType(), True),    # "CRSDepTime":"2015-12-31T03:05:00.000-08:00"
StructField("Carrier", StringType(), True),     # "Carrier":"WN"
StructField("DayOfMonth", IntegerType(), True), # "DayOfMonth":31
StructField("DayOfWeek", IntegerType(), True),  # "DayOfWeek":4
StructField("DayOfYear", IntegerType(), True),  # "DayOfYear":365
StructField("DepDelay", DoubleType(), True),     # "DepDelay":14.0
StructField("Dest", StringType(), True),        # "Dest":"SAN"
StructField("Distance", DoubleType(), True),     # "Distance":368.0
StructField("FlightDate", DateType(), True),    # "FlightDate":"2015-12-30T16:00:00.000-08:00"
StructField("FlightNum", StringType(), True),   # "FlightNum":"6109"
StructField("Origin", StringType(), True),      # "Origin":"TUS"
])

input_path = "{}/Agile_Data_Code_2/data/simple_flight_delay_features.jsonl.bz2".format(
base_path
)
features = spark.read.json(input_path, schema=schema)
features.first()




Row(ArrDelay=13.0, CRSArrTime=datetime.datetime(2015, 1, 1, 13, 10), CRSDepTime=datetime.datetime(2015, 1, 1, 10, 30), Carrier='AA', DayOfMonth=1, DayOfWeek=4, DayOfYear=1, DepDelay=14.0, Dest='DFW', Distance=569.0, FlightDate=datetime.date(2014, 12, 31), FlightNum='1024', Origin='ABQ')

In [9]:
#
# Check for nulls in features before using Spark ML
#
null_counts = [(column, features.where(features[column].isNull()).count()) for column in features.columns]
cols_with_nulls = filter(lambda x: x[1] > 0, null_counts)
print(list(cols_with_nulls))

[]


In [10]:
#
# Add a Route variable to replace FlightNum
#
from pyspark.sql.functions import lit, concat
features_with_route = features.withColumn(
'Route',
concat(
  features.Origin,
  lit('-'),
  features.Dest
)
)
features_with_route.show(6)

+--------+--------------------+--------------------+-------+----------+---------+---------+--------+----+--------+----------+---------+------+-------+
|ArrDelay|          CRSArrTime|          CRSDepTime|Carrier|DayOfMonth|DayOfWeek|DayOfYear|DepDelay|Dest|Distance|FlightDate|FlightNum|Origin|  Route|
+--------+--------------------+--------------------+-------+----------+---------+---------+--------+----+--------+----------+---------+------+-------+
|    13.0|2015-01-01 13:10:...|2015-01-01 10:30:...|     AA|         1|        4|        1|    14.0| DFW|   569.0|2014-12-31|     1024|   ABQ|ABQ-DFW|
|    17.0|2015-01-01 05:15:...|2015-01-01 02:25:...|     AA|         1|        4|        1|    14.0| DFW|   569.0|2014-12-31|     1184|   ABQ|ABQ-DFW|
|    36.0|2015-01-01 06:45:...|2015-01-01 04:00:...|     AA|         1|        4|        1|    -2.0| DFW|   569.0|2014-12-31|      336|   ABQ|ABQ-DFW|
|   -21.0|2015-01-01 14:30:...|2015-01-01 12:55:...|     AA|         1|        4|        1|   

In [36]:
#
# Use pysmark.ml.feature.Bucketizer to bucketize ArrDelay into on-time, slightly late, very late (0, 1, 2)
#
from pyspark.ml.feature import Bucketizer

# Setup the Bucketizer
splits = [-float("inf"), -15.0, 0, 30.0, float("inf")]
arrival_bucketizer = Bucketizer(
splits=splits,
inputCol="ArrDelay",
outputCol="ArrDelayBucket"
)


# Apply the bucketizer
ml_bucketized_features = arrival_bucketizer.transform(features_with_route)
ml_bucketized_features.select("ArrDelay", "ArrDelayBucket").show()

+--------+--------------+
|ArrDelay|ArrDelayBucket|
+--------+--------------+
|    13.0|           2.0|
|    17.0|           2.0|
|    36.0|           3.0|
|   -21.0|           0.0|
|   -14.0|           1.0|
|    16.0|           2.0|
|    -7.0|           1.0|
|    13.0|           2.0|
|    25.0|           2.0|
|    58.0|           3.0|
|    14.0|           2.0|
|     1.0|           2.0|
|   -29.0|           0.0|
|   -10.0|           1.0|
|    -3.0|           1.0|
|    -8.0|           1.0|
|    -1.0|           1.0|
|   -14.0|           1.0|
|   -16.0|           0.0|
|    18.0|           2.0|
+--------+--------------+
only showing top 20 rows



In [37]:
ml_bucketized_features

DataFrame[ArrDelay: double, CRSArrTime: timestamp, CRSDepTime: timestamp, Carrier: string, DayOfMonth: int, DayOfWeek: int, DayOfYear: int, DepDelay: double, Dest: string, Distance: double, FlightDate: date, FlightNum: string, Origin: string, Route: string, ArrDelayBucket: double]

In [39]:
#
# Extract features tools in with pyspark.ml.feature
#
from pyspark.ml.feature import StringIndexer, VectorAssembler

# Turn category fields into indexes
for column in ["Carrier", "Origin", "Dest", "Route"]:
    print(column)
    string_indexer = StringIndexer(
          inputCol=column,
          outputCol=column + "_index"
        )

    string_indexer_model = string_indexer.fit(ml_bucketized_features)
    ml_bucketized_features = string_indexer_model.transform(ml_bucketized_features)

    # Drop the original column
    ml_bucketized_features = ml_bucketized_features.drop(column)
    
    # Save the pipeline model
    string_indexer_output_path = "{}/models/string_indexer_model_{}.bin".format(
      base_path,
      column
    )
    string_indexer_model.write().overwrite().save(string_indexer_output_path)
    
    


Carrier
Origin
Dest
Route


In [32]:
# Combine continuous, numeric fields with indexes of nominal ones
# ...into one feature vector
numeric_columns = [
"DepDelay", "Distance",
"DayOfMonth", "DayOfWeek",
"DayOfYear"]
index_columns = ["Carrier_index", "Origin_index",
               "Dest_index", "Route_index"]
vector_assembler = VectorAssembler(
inputCols=numeric_columns + index_columns,
outputCol="Features_vec"
)
final_vectorized_features = vector_assembler.transform(ml_bucketized_features)


# Drop the index columns
for column in index_columns:
    final_vectorized_features = final_vectorized_features.drop(column)

# Inspect the finalized features
final_vectorized_features.show()

+--------+--------------------+--------------------+----------+---------+---------+--------+--------+----------+---------+--------------+--------------------+
|ArrDelay|          CRSArrTime|          CRSDepTime|DayOfMonth|DayOfWeek|DayOfYear|DepDelay|Distance|FlightDate|FlightNum|ArrDelayBucket|        Features_vec|
+--------+--------------------+--------------------+----------+---------+---------+--------+--------+----------+---------+--------------+--------------------+
|    13.0|2015-01-01 13:10:...|2015-01-01 10:30:...|         1|        4|        1|    14.0|   569.0|2014-12-31|     1024|           2.0|[14.0,569.0,1.0,4...|
|    17.0|2015-01-01 05:15:...|2015-01-01 02:25:...|         1|        4|        1|    14.0|   569.0|2014-12-31|     1184|           2.0|[14.0,569.0,1.0,4...|
|    36.0|2015-01-01 06:45:...|2015-01-01 04:00:...|         1|        4|        1|    -2.0|   569.0|2014-12-31|      336|           3.0|[-2.0,569.0,1.0,4...|
|   -21.0|2015-01-01 14:30:...|2015-01-01 12:5

In [33]:
# Instantiate and fit random forest classifier on all the data
from pyspark.ml.classification import RandomForestClassifier
rfc = RandomForestClassifier(
featuresCol="Features_vec",
labelCol="ArrDelayBucket",
predictionCol="Prediction",
maxBins=4657,
maxMemoryInMB=1024
)
model = rfc.fit(final_vectorized_features)

In [34]:
# Evaluate model using test data
predictions = model.transform(final_vectorized_features)

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
predictionCol="Prediction",
labelCol="ArrDelayBucket",
metricName="accuracy"
)
accuracy = evaluator.evaluate(predictions)
print("Accuracy = {}".format(accuracy))

# Check the distribution of predictions
predictions.groupBy("Prediction").count().show()

# Check a sample
predictions.sample(False, 0.001, 18).orderBy("CRSDepTime").show(6)

Accuracy = 0.5948777110567574
+----------+-------+
|Prediction|  count|
+----------+-------+
|       0.0|    139|
|       1.0|4158208|
|       3.0| 582953|
|       2.0| 972708|
+----------+-------+

+--------+--------------------+--------------------+----------+---------+---------+--------+--------+----------+---------+--------------+--------------------+--------------------+--------------------+----------+
|ArrDelay|          CRSArrTime|          CRSDepTime|DayOfMonth|DayOfWeek|DayOfYear|DepDelay|Distance|FlightDate|FlightNum|ArrDelayBucket|        Features_vec|       rawPrediction|         probability|Prediction|
+--------+--------------------+--------------------+----------+---------+---------+--------+--------+----------+---------+--------------+--------------------+--------------------+--------------------+----------+
|   -25.0|2015-01-01 06:07:...|2015-01-01 02:30:...|         1|        4|        1|    -4.0|  1237.0|2014-12-31|      569|           0.0|[-4.0,1237.0,1.0,...|[6.4799

In [35]:
# Save the pipeline model
string_indexer_output_path = "{}/models/string_indexer_model_{}.bin".format(
  base_path,
  column
)
string_indexer_model.write().overwrite().save(string_indexer_output_path)
# Save the numeric vector assembler
vector_assembler_path = "{}/models/numeric_vector_assembler.bin".format(base_path)
vector_assembler.write().overwrite().save(vector_assembler_path)

# Save the bucketizer
arrival_bucketizer_path = "{}/models/arrival_bucketizer_2.0.bin".format(base_path)
arrival_bucketizer.write().overwrite().save(arrival_bucketizer_path)
# Save the new model over the old one
model_output_path = "{}/models/spark_random_forest_classifier.flight_delays.5.0.bin".format(base_path)
model.write().overwrite().save(model_output_path)